<div style="
    background: linear-gradient(90deg, #ff6a00, #ff3e3e, #ff0099, #9b00ff, #3b82f6);
    padding: 18px;
    border-radius: 12px;
    color: white;
    font-size:17px;
    line-height:1.55;
    box-shadow:0 3px 10px rgba(0,0,0,0.25);
">
  This notebook demonstrates a real-time Vehicle Tracking & Counting pipeline using YOLO11, OpenCV, and Python.  
  It detects and tracks multiple vehicle classes with unique IDs, draws bounding boxes, labels, and centroids,  
  and counts vehicles reliably when they cross a virtual line.  
  Finally, it exports a fully annotated output video, making the workflow practical for traffic monitoring and analytics.
</div>


<div style="
    margin-top:18px;
    display:flex;
    gap:15px;
    flex-wrap:wrap;
    justify-content:center;
">

  <!-- Kaggle -->
  <a href="https://www.kaggle.com/mdferozahmedafm" target="_blank"
     style="
        text-decoration:none;
        background:linear-gradient(90deg,#0085ff,#00c2ff);
        padding:12px 24px;
        border-radius:12px;
        color:white;
        font-weight:700;
        font-size:16px;
        box-shadow:0 3px 8px rgba(0,0,0,0.25);
     ">
     Kaggle
  </a>

  <!-- GitHub -->
  <a href="https://github.com/feroz-afm" target="_blank"
     style="
        text-decoration:none;
        background:linear-gradient(90deg,#333,#000);
        padding:12px 24px;
        border-radius:12px;
        color:white;
        font-weight:700;
        font-size:16px;
        box-shadow:0 3px 8px rgba(0,0,0,0.25);
     ">
     GitHub 
  </a>

  <!-- Coursera -->
  <a href="https://www.coursera.org/account-profile" target="_blank"
     style="
        text-decoration:none;
        background:linear-gradient(90deg,#ff6a00,#ff3e3e);
        padding:12px 24px;
        border-radius:12px;
        color:white;
        font-weight:700;
        font-size:16px;
        box-shadow:0 3px 8px rgba(0,0,0,0.25);
     ">
     Coursera 
  </a>

</div>


<div style="background-color:orange; padding:10px; border-radius:8px; text-align:center;">
  <h1 style="margin:0; color:#333;"> Load DATA  </h1>
</div>


In [ ]:


import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



<div style="background-color:white; padding:10px; border-radius:8px; text-align:center;">
  <h1 style="margin:0; color:#333;"> Import Libraries </h1>
</div>


In [ ]:
!pip -q install -U ultralytics opencv-python pyyaml


<div style="background-color:yellow; padding:10px; border-radius:8px; text-align:center;">
  <h2 style="margin:0; color:#333;"> Configuration & Model Setup </h2>
</div>


In [ ]:
import os, yaml, cv2
from collections import defaultdict
from ultralytics import YOLO

VIDEO_IN  = "/kaggle/input/vehicle-dataset-sample-feroz/Vehicle Dataset Sample (Feroz).mp4"
VIDEO_OUT = "/kaggle/working/vehicle_tracking_counting.mp4"


MODEL_WEIGHTS = "yolo11m.pt"   


CONF_THRES = 0.10
IOU_THRES  = 0.50
IMGSZ      = 1280   

ALLOWED = {"bicycle","car","motorcycle","bus","truck"}


TRACK_COLOR = (0,255,255)  # yellow
COUNT_COLOR = (0,255,0)    # green
LINE_COLOR  = (0,0,255)    # red

# Text sizes
FONT = cv2.FONT_HERSHEY_SIMPLEX
ID_SCALE, ID_THICK = 0.55, 2
COUNT_SCALE, COUNT_THICK = 0.90, 2


custom_tracker = "/kaggle/working/bytetrack_low.yaml"
cfg = {
    "tracker_type": "bytetrack",
    "track_high_thresh": 0.15,
    "track_low_thresh": 0.05,
    "new_track_thresh": 0.12,
    "track_buffer": 90,
    "match_thresh": 0.8,
    "fuse_score": True
}
with open(custom_tracker, "w") as f:
    yaml.safe_dump(cfg, f)

model = YOLO(MODEL_WEIGHTS)
print("Tracker:", custom_tracker)


<div style="background-color:lime; padding:10px; border-radius:8px; text-align:center;">
  <h2 style="margin:0; color:#333;"> Vehicle Tracking, Line Counting & Save Output </h2>
</div>


In [ ]:
cap = cv2.VideoCapture(VIDEO_IN)
if not cap.isOpened():
    raise FileNotFoundError("Cannot open video")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 25

LINE_Y = int(h * 0.62)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

prev_y = {}
counted_ids = set()
counts = defaultdict(int)

while True:
    ok, frame = cap.read()
    if not ok:
        break


    r = model.track(
        frame,
        persist=True,
        tracker=custom_tracker,
        conf=CONF_THRES,
        iou=IOU_THRES,
        imgsz=IMGSZ,
        verbose=False
    )[0]

    # red line
    cv2.line(frame, (0, LINE_Y), (w, LINE_Y), LINE_COLOR, 3)

    if r.boxes is not None and r.boxes.id is not None:
        boxes = r.boxes.xyxy.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)
        ids   = r.boxes.id.cpu().numpy().astype(int)

        for (x1,y1,x2,y2), c, tid in zip(boxes, clss, ids):
            name = model.names.get(int(c), str(c))
            if name not in ALLOWED:
                continue

            x1,y1,x2,y2 = map(int, [x1,y1,x2,y2])
            cx = (x1+x2)//2
            cy = (y1+y2)//2

          
            cv2.rectangle(frame, (x1,y1), (x2,y2), TRACK_COLOR, 2)
            cv2.circle(frame, (cx,cy), 4, TRACK_COLOR, -1)

           
            label = f"ID:{tid} {name}"
            (tw, th), _ = cv2.getTextSize(label, FONT, ID_SCALE, ID_THICK)
            y_text = max(20, y1 - 8)
            cv2.rectangle(frame, (x1, y_text - th - 6), (x1 + tw + 6, y_text + 4), (0,0,0), -1)
            cv2.putText(frame, label, (x1+3, y_text), FONT, ID_SCALE, TRACK_COLOR, ID_THICK, cv2.LINE_AA)

          
            py = prev_y.get(tid)
            if py is not None:
                crossed = (py < LINE_Y <= cy) or (py > LINE_Y >= cy)
                if crossed and tid not in counted_ids:
                    counted_ids.add(tid)
                    counts[name] += 1
            prev_y[tid] = cy

   
    y0 = 40
    for k in ["bicycle","bus","car","motorcycle","truck"]:
        cv2.putText(frame, f"{k}: {counts[k]}", (20, y0),
                    FONT, COUNT_SCALE, COUNT_COLOR, COUNT_THICK, cv2.LINE_AA)
        y0 += 35

    writer.write(frame)

cap.release()
writer.release()
print("Saved:", VIDEO_OUT)


<div style="
    background: linear-gradient(90deg, #ff6a00, #ff3e3e, #ff0099, #9b00ff, #3b82f6);
    padding: 18px;
    border-radius: 12px;
    color: white;
    font-size:17px;
    line-height:1.55;
    box-shadow:0 3px 10px rgba(0,0,0,0.25);
">
  After saving the annotated output video, we convert it to an H.264 (libx264) MP4 format for better browser compatibility.  
  This helps Kaggle render the video inline (instead of forcing a download) and ensures smoother playback in the notebook.  
  Finally, we preview the converted video directly and generate a download link for sharing on platforms like LinkedIn.
</div>


In [ ]:
!ffmpeg -y -i "/kaggle/working/vehicle_tracking_counting.mp4" \
  -vcodec libx264 -pix_fmt yuv420p -movflags +faststart -an \
  "/kaggle/working/vehicle_tracking_counting_h264.mp4"


In [ ]:
from IPython.display import Video
Video("/kaggle/working/vehicle_tracking_counting_h264.mp4", embed=True, width=900)
